In [1]:
from pathlib import Path
import torch
import torch.nn as nn
from loguru import logger
import warnings
warnings.simplefilter("ignore", UserWarning)

Let's use the mads_datasets package (see [github](https://github.com/raoulg/mads_datasets) for more details) which I created for these lessons to give everyone easy access to the datasets we use for training.

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

for dataset in DatasetType:
    print(dataset)

DatasetType.FLOWERS
DatasetType.IMDB
DatasetType.GESTURES
DatasetType.FASHION
DatasetType.SUNSPOTS
DatasetType.IRIS
DatasetType.PENGUINS
DatasetType.FAVORITA
DatasetType.SECURE


There are a few datasets. For images, we can use FLOWERS (~3000 photos of flowers in 5 categories) and FASHION (60k fashion icons 28x28 pixels big).

Lets start with our good'ol MNIST.

In [3]:

fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
batchsize = 64
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2024-11-17 11:06:33.665 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\dilek\.cache\mads_datasets\fashionmnist
2024-11-17 11:06:33.666 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\dilek\.cache\mads_datasets\fashionmnist\fashionmnist.pt


We can obtain an item:

In [4]:
x, y = next(iter(trainstreamer))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

The image follows the channels-first convention: (channel, width, height). The label is an integer.

Let's re-use the model we had:

In [5]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cpu


In [6]:
from torch import nn
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self, filters, units1, units2, input_size=(32, 1, 28, 28)):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filters, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        activation_map_size = self._conv_test(input_size)
        logger.info(f"Aggregating activationmap with size {activation_map_size}")
        self.agg = nn.AvgPool2d(activation_map_size)

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, 10)
        )

    def _conv_test(self, input_size = (32, 1, 28, 28)):
        x = torch.ones(input_size)
        x = self.convolutions(x)
        return x.shape[-2:]

    def forward(self, x):
        x = self.convolutions(x)
        x = self.agg(x)
        logits = self.dense(x)
        return logits

model = CNN(filters=32, units1=128, units2=64).to("cpu")

2024-11-17 11:06:49.595 | INFO     | __main__:__init__:22 - Aggregating activationmap with size torch.Size([2, 2])


Using cpu device


In [7]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 12, 12]           9,248
              ReLU-5           [-1, 32, 12, 12]               0
         MaxPool2d-6             [-1, 32, 6, 6]               0
            Conv2d-7             [-1, 32, 4, 4]           9,248
              ReLU-8             [-1, 32, 4, 4]               0
         MaxPool2d-9             [-1, 32, 2, 2]               0
        AvgPool2d-10             [-1, 32, 1, 1]               0
          Flatten-11                   [-1, 32]               0
           Linear-12                  [-1, 128]           4,224
             ReLU-13                  [-1, 128]               0
           Linear-14                   

And set up the optimizer, loss and accuracy.

In [8]:
import torch.optim as optim
from mltrainer import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [9]:
yhat = model(x.to("cpu"))
accuracy(y.to("cpu"), yhat)

tensor(0.1250)

# MLflow
MLflow is an open-source platform designed to manage the entire Machine Learning (ML) lifecycle, including experimentation, reproducibility, deployment, and governance. It provides a set of APIs and tools to streamline ML workflows, making it easier to track experiments, package code, manage model versions, and deploy models.

Reasons to use MLflow over TensorBoard, gin-config, or Ray:

- End-to-end ML lifecycle management: While TensorBoard focuses on visualizing model training metrics and gin-config on hyperparameter configuration, MLflow covers a broader range of tasks, such as experiment tracking, model packaging, and deployment.

- Framework agnostic: MLflow is not tied to a specific ML framework, making it suitable for projects using different libraries or even multiple libraries.

- Model Registry: MLflow provides a centralized model registry, allowing you to version, track, and manage your models, which is not available in TensorBoard or gin-config.

- Deployment support: MLflow facilitates model deployment to various platforms, such as local, cloud, or Kubernetes environments, whereas TensorBoard and gin-config are not built for deployment tasks.

- Integration with other tools: MLflow integrates with popular tools and platforms like Databricks, AWS, and Azure, making it easy to incorporate into existing workflows.

However, the choice between MLflow and other tools like TensorBoard, gin-config, or Ray depends on your specific use case and the scope of the ML workflow you want to manage.

In [10]:
experiment_path = "mlflow_test"

In [11]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_path)

<Experiment: artifact_location='file:///C:/Users/dilek/Desktop/Advanced_AI_Applications_WS24-25_MADS_HSRW/notebooks/2_convolutions/mlruns/1', creation_time=1730121984736, experiment_id='1', last_update_time=1730121984736, lifecycle_stage='active', name='mlflow_test', tags={}>

In the code above, we set the MLflow tracking URI to a local SQLite database file. This is done to configure the storage location for MLflow's experiment tracking data, such as metrics, parameters, and artifacts. By specifying a SQLite database, we enable a lightweight and easy-to-use storage solution for tracking the experiments and their associated information.

The line mlflow.set_experiment("mnist_convolutions") sets the active MLflow experiment to "mnist_convolutions". This is useful for organizing and grouping your runs, as it allows you to associate the upcoming ML training runs with a specific experiment name, making it easier to search, compare, and analyze the results later.

In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

We import functions and classes from the hyperopt library to perform hyperparameter optimization. This library helps us find the best hyperparameter values for our machine learning model by searching through a defined search space and using optimization algorithms like Tree-structured Parzen Estimator (TPE). The goal is to improve our model's performance by tuning its hyperparameters.

Advantages of TPE:

- Model-based approach: TPE is a Bayesian optimization method that models the objective function as a probability distribution. It learns from previous evaluations to decide which points in the search space to explore next, making it more efficient in finding optimal hyperparameters.

- Exploration-exploitation trade-off: TPE balances the trade-off between exploration (searching in new regions of the search space) and exploitation (refining around the current best points). This can lead to better results in problems with complex search spaces.

- Continuous hyperparameter optimization: TPE can handle continuous hyperparameters more naturally, as it builds a probability model to estimate the performance for any given point in the search space.

Lets set up an objective function and start logging some usefull things we might want to track:

In [14]:
modeldir = Path("../../models/mnist").resolve()
if not modeldir.exists():
    modeldir.mkdir()
    print(f"Created {modeldir}")

In [15]:
import torch.optim as optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from datetime import datetime

# Define the hyperparameter search space
settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW],
)


# Define the objective function for hyperparameter optimization
def objective(params):
    # Start a new MLflow run for tracking the experiment
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", "convnet")
        mlflow.set_tag("dev", "raoul")
        # Log hyperparameters to MLflow
        mlflow.log_params(params)
        mlflow.log_param("batchsize", f"{batchsize}")


        # Initialize the optimizer, loss function, and accuracy metric
        optimizer = optim.Adam
        loss_fn = torch.nn.CrossEntropyLoss()
        accuracy = metrics.Accuracy()

        # Instantiate the CNN model with the given hyperparameters
        model = CNN(**params)
        # Train the model using a custom train loop
        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optimizer,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        # Save the trained model with a timestamp
        tag = datetime.now().strftime("%Y%m%d-%H%M")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

        # Log the saved model as an artifact in MLflow
        mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")
        return {'loss' : trainer.test_loss, 'status': STATUS_OK}

In [16]:
search_space = {
    'filters' : scope.int(hp.quniform('filters', 16, 128, 8)),
    'units1' : scope.int(hp.quniform('units1', 32, 128, 8)),
    'units2' : scope.int(hp.quniform('units2', 32, 128, 8)),
}

We define a search space for hyperparameter optimization using Hyperopt. The search space specifies the range and distribution of hyperparameters to explore during the optimization process. This is crucial for finding the optimal set of hyperparameters that yield the best performance for the machine learning model. The search space defined here includes the number of filters in the convolutional layers, and the number of units in two fully connected layers, allowing Hyperopt to find the best combination within the given ranges.


Now, finally, let us perform the hyperparameter search using the fmin function from hyperopt. The function takes the following arguments:

- `fn=objective`: The objective function to minimize, which is defined earlier to train the model and return the test loss.
- `space=search_space`: The search space defined earlier, containing the range of hyperparameters to explore.
- `algo=tpe.suggest`: The optimization algorithm to use, in this case, the Tree-structured Parzen Estimator (TPE) method.
- `max_evals=10`: The maximum number of function evaluations, i.e., the maximum number of hyperparameter combinations to try.
- `trials=Trials()`: A Trials object to store the results of each evaluation.

The fmin function searches for the best hyperparameters within the given search space using the TPE algorithm, aiming to minimize the objective function (test loss). Once the optimization process is completed, the best hyperparameters found are stored in the best_result variable.

In [17]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

  0%|                                                                            | 0/3 [00:00<?, ?trial/s, best loss=?]

2024-11-17 11:08:34.402 | INFO     | __main__:__init__:22 - Aggregating activationmap with size torch.Size([2, 2])
2024-11-17 11:08:34.404 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog\20241117-110834
2024-11-17 11:08:36.187 | INFO     | mltrainer.trainer:__init__:70 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  5%|####1                                                                             | 5/100 [00:00<00:02, 45.40it/s]

 10%|########1                                                                        | 10/100 [00:00<00:02, 33.56it/s]

 17%|#############7                                                                   | 17/100 [00:00<00:01, 45.26it/s]

 25%|#############

 33%|████████████████▋                                 | 1/3 [00:10<00:20, 10.36s/trial, best loss: 0.8096083533763886]

2024-11-17 11:08:44.533 | INFO     | __main__:__init__:22 - Aggregating activationmap with size torch.Size([2, 2])
2024-11-17 11:08:44.533 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog\20241117-110844
2024-11-17 11:08:44.537 | INFO     | mltrainer.trainer:__init__:70 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  5%|####1                                                                             | 5/100 [00:00<00:02, 44.44it/s]

 11%|########9                                                                        | 11/100 [00:00<00:01, 49.23it/s]

 16%|############9                                                                    | 16/100 [00:00<00:01, 44.76it/s]

 21%|#############

 67%|██████████████████████████████████▋                 | 2/3 [00:20<00:09,  9.96s/trial, best loss: 0.80675312936306]

2024-11-17 11:08:54.175 | INFO     | __main__:__init__:22 - Aggregating activationmap with size torch.Size([2, 2])
2024-11-17 11:08:54.177 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog\20241117-110854
2024-11-17 11:08:54.179 | INFO     | mltrainer.trainer:__init__:70 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  6%|####9                                                                             | 6/100 [00:00<00:01, 57.53it/s]

 12%|#########7                                                                       | 12/100 [00:00<00:02, 43.68it/s]

 19%|###############3                                                                 | 19/100 [00:00<00:01, 52.74it/s]

 25%|#############

100%|████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.21s/trial, best loss: 0.80675312936306]


After running this, you can look at the best_result

In [18]:
best_result

{'filters': 32.0, 'units1': 56.0, 'units2': 120.0}

But you can also explore the UI from mlflow. It is pretty nice. The help you out, you can use the makefile by first navigating to `/notebooks/2_convolutions` in the terminal and then typing `make show_logs`. This starts a server you can open at `localhost:5000` . Also, have a look at the `Makefile` in this folder to see what you execute. It save the user from typing an inconvenient long and complex command every time.